In [1]:
%pip install torch highway-env

You should consider upgrading via the '/Users/iato/Code/autobots/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import gymnasium as gym
import highway_env
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mp

import env_exec

In [ ]:
env = gym.make(
    "highway-fast-v0",
    render_mode="rgb_array",
    config={
        "action": {
            "type": "DiscreteMetaAction",
        },
        "observation": {
            "type": "LidarObservation",
            "cells": 128,
            "maximum_range": 64,
            "normalise": True,
        },
        "vehicles_count": 20,
    },
)

epochs = 20
episodes = 1000
epsilon = 0.2
episilon_decay = 0.99
hidden_size = 512
learning_rate = 0.05
momentum = 0.9
test_episodes = 100

obs, info = env.reset()

/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")
device

device(type='mps')

In [5]:
flattened_observation_size = np.prod(obs.shape)
net = torch.nn.Sequential(
    torch.nn.Linear(flattened_observation_size, hidden_size),
    torch.nn.LeakyReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(hidden_size, hidden_size),
    torch.nn.LeakyReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(hidden_size, env.action_space.n),
).to(device)

In [6]:
optimizer = torch.optim.Adadelta(net.parameters(), lr=learning_rate, rho=momentum)
loss_fn = torch.nn.MSELoss()

In [ ]:

for epoch in range(epochs):
    # Parallelize the execution of the episodes
    mpPool = mp.Pool(mp.cpu_count() * 2)
    results = mpPool.map(env_exec.run_episode, range(episodes))
    mpPool.close()

    train_losses = []
    test_losses = []
    rewards = []
    
    net.train()
    # Update the network with the results of the episodes
    for observations, action_rewards in results:
        for obs, action_reward in zip(observations, action_rewards):
            x = torch.tensor(obs, dtype=torch.float32).flatten().to(device)
            y = torch.tensor(action_reward, dtype=torch.float32).to(device)

            optimizer.zero_grad()
            y_pred = net(x)
            loss = loss_fn(y_pred, y)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
    
    # Test the network
    net.eval()
    for episode in range(test_episodes):
        obs, info = env.reset(seed=episode)
        done, truncated = False, False
        total_reward = 0
        while not done and not truncated:
            x = torch.tensor(obs, dtype=torch.float32).flatten().to(device)
            y_pred = net(x)
            action = max(enumerate(y_pred), key=lambda x: x[1])[0]
            nobs, reward, done, truncated, info = env.step(action)
            
            y = y_pred.clone()
            y[action] = torch.tensor(reward, dtype=torch.float32).to(device)
            
            total_reward += reward
            test_losses.append(loss_fn(y_pred, y).item())

            obs = nobs

            # env.render()
        rewards.append(total_reward)



    avg_train_loss = (sum(train_losses)) / (len(train_losses))
    avg_test_loss = (sum(test_losses)) / (len(test_losses))
    avg_reward = (sum(rewards)) / (len(rewards))

    print(f"Epoch {epoch}\t Test Loss: {avg_test_loss}\t Train Loss: {avg_train_loss}\t Reward: {avg_reward}")
    torch.save(net.state_dict(), "model.pth")


/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/highway_env/utils.py:402: RuntimeWarning: divide by zero encountered in scalar divide
  interval_1 = [(a - r) @ u / rqu, (b - r) @ u / rqu]
/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/highway_env/utils.py:403: RuntimeWarning: divide by zero encountered in scalar divide
  interval_2 = [(a - r) @ v / rqv, (d - r) @ v / rqv]
/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/highway_env/utils.py:402: RuntimeWarning: divide by zero encountered in scalar divide
  interval_1 = [(a - r) @ u / rqu, (b - r) @ u / rqu]
/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/highway_env/utils.py:403: RuntimeWarning: divide by zero encountered in scalar divide
  interval_2 = [(a - r) @ v / rqv, (d - r) @ v / rqv]
/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to b

Epoch 0	 Test Loss: 0.03212191512460237	 Train Loss: 0.024260283250679665	 Reward: 7.080000270090111


/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/highway_env/utils.py:402: RuntimeWarning: divide by zero encountered in scalar divide
  interval_1 = [(a - r) @ u / rqu, (b - r) @ u / rqu]
/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/Users/iato/Code/autobots/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/iato/Code/autobots/.venv/lib/python3.9